In [2]:
!pip install -e git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper

Obtaining bq_helper from git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper
  Cloning https://github.com/SohierDane/BigQuery_Helper to /src/bq-helper
  Running command git clone --filter=blob:none --quiet https://github.com/SohierDane/BigQuery_Helper /src/bq-helper
  Resolved https://github.com/SohierDane/BigQuery_Helper to commit 8615a7f6c1663e7f2d48aa2b32c2dbcb600a440f
  Preparing metadata (setup.py) ... done
  DEPRECATION: Legacy editable install of bq_helper from git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop fo

In [8]:
!pip install db-dtypes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 96.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 2.0.0
    Uninstalling pyarrow-2.0.0:
      Successfully uninstalled pyarrow-2.0.0


In [1]:
#Install above packages, restart kernel and then run the remaining code.
#Kernel crashes sometimes so may have to run in parts - 1 response at a time.

In [2]:
import bq_helper
from bq_helper import BigQueryHelper
stackOverflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",dataset_name="stackoverflow")

In [3]:
bq_assistant = BigQueryHelper("bigquery-public-data", "stackoverflow")
bq_assistant.list_tables()

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [5]:
head = bq_assistant.head("stackoverflow_posts", num_rows=20)
head.head()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,376158,None,<p>Assuming // do something is more than just ...,None,None,0,None,2008-12-17 21:52:23.097000+00:00,None,2008-12-17 21:52:23.097000+00:00,NaT,None,NaN,n8wrl,37710.0,361115.0,2,0,,None
1,171458,None,<p>A test is a way of precisely describing som...,None,None,0,None,2008-10-05 04:00:07+00:00,None,2008-10-05 04:00:07+00:00,NaT,None,NaN,jml,25221.0,135789.0,2,2,,None
2,11421,None,<p>You need to create your own exchange messag...,None,None,0,None,2008-08-14 17:59:49.753000+00:00,None,2008-08-14 18:21:15.763000+00:00,2008-08-14 18:21:15.763000+00:00,Kevin Kenny,419.0,Kevin Kenny,419.0,11275.0,2,2,,None
3,602817,None,<p>I create UI paper prototypes when designing...,None,None,0,None,2009-03-02 15:48:55.137000+00:00,None,2009-03-02 15:48:55.137000+00:00,NaT,None,NaN,Esko Luontola,62130.0,602708.0,2,1,,None
4,442925,None,<p>We are experimenting with switching from De...,None,None,0,None,2009-01-14 13:27:54.103000+00:00,None,2009-01-14 13:27:54.103000+00:00,NaT,None,NaN,Ian Horwill,5816.0,141435.0,2,0,,None


### Fetch data from BigQuery and store as Parquet files

In [20]:
query1 = """SELECT posts.id AS post_id,
posts.title AS post_title,
posts.body AS post_body,
posts.accepted_answer_id AS post_accepted_answer_id,
posts.answer_count AS post_answer_count,
posts.comment_count AS post_comment_count,
posts.community_owned_date AS post_community_owned_date,
posts.creation_date AS post_creation_date,
posts.favorite_count AS post_favorite_count,
posts.last_activity_date AS post_last_activity_date,
posts.last_edit_date AS post_last_edit_date,
posts.last_editor_display_name AS post_last_editor_display_name,
posts.last_editor_user_id AS post_last_editor_user_id,
posts.owner_display_name AS post_owner_display_name,
posts.owner_user_id AS post_owner_user_id,
posts.parent_id AS post_parent_id,
posts.post_type_id AS post_post_type_id,
posts.score AS post_score,
posts.tags AS post_tags,
posts.view_count AS post_view_count,
answers.id AS answer_id,
answers.title AS answer_title,
answers.body AS answer_body,
answers.accepted_answer_id AS answer_accepted_answer_id,
answers.answer_count AS answer_answer_count,
answers.comment_count AS answer_comment_count,
answers.community_owned_date AS answer_community_owned_date,
answers.creation_date AS answer_creation_date,
answers.favorite_count AS answer_favorite_count,
answers.last_activity_date AS answer_last_activity_date,
answers.last_edit_date AS answer_last_edit_date,
answers.last_editor_display_name AS answer_last_editor_display_name,
answers.last_editor_user_id AS answer_last_editor_user_id,
answers.owner_display_name AS answer_owner_display_name,
answers.owner_user_id AS answer_owner_user_id,
answers.parent_id AS answer_parent_id,
answers.post_type_id AS answer_post_type_id,
answers.score AS answer_score,
answers.tags AS answer_tags,
answers.view_count AS answer_view_count
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` posts
LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` answers
ON posts.id = answers.parent_id
WHERE EXTRACT(YEAR FROM posts.creation_date) between 2008 and 2009 AND posts.answer_count >=5;
"""
response1 = stackOverflow.query_to_pandas_safe(query1,max_gb_scanned=70)

In [21]:
response1.shape

(779440, 40)

In [10]:
import pyarrow 
import pyarrow.parquet 

In [11]:
response1.to_parquet('StackOverflow_test_3.parquet')  

In [17]:
query2 = """SELECT posts.id AS post_id,
posts.title AS post_title,
posts.body AS post_body,
posts.accepted_answer_id AS post_accepted_answer_id,
posts.answer_count AS post_answer_count,
posts.comment_count AS post_comment_count,
posts.community_owned_date AS post_community_owned_date,
posts.creation_date AS post_creation_date,
posts.favorite_count AS post_favorite_count,
posts.last_activity_date AS post_last_activity_date,
posts.last_edit_date AS post_last_edit_date,
posts.last_editor_display_name AS post_last_editor_display_name,
posts.last_editor_user_id AS post_last_editor_user_id,
posts.owner_display_name AS post_owner_display_name,
posts.owner_user_id AS post_owner_user_id,
posts.parent_id AS post_parent_id,
posts.post_type_id AS post_post_type_id,
posts.score AS post_score,
posts.tags AS post_tags,
posts.view_count AS post_view_count,
answers.id AS answer_id,
answers.title AS answer_title,
answers.body AS answer_body,
answers.accepted_answer_id AS answer_accepted_answer_id,
answers.answer_count AS answer_answer_count,
answers.comment_count AS answer_comment_count,
answers.community_owned_date AS answer_community_owned_date,
answers.creation_date AS answer_creation_date,
answers.favorite_count AS answer_favorite_count,
answers.last_activity_date AS answer_last_activity_date,
answers.last_edit_date AS answer_last_edit_date,
answers.last_editor_display_name AS answer_last_editor_display_name,
answers.last_editor_user_id AS answer_last_editor_user_id,
answers.owner_display_name AS answer_owner_display_name,
answers.owner_user_id AS answer_owner_user_id,
answers.parent_id AS answer_parent_id,
answers.post_type_id AS answer_post_type_id,
answers.score AS answer_score,
answers.tags AS answer_tags,
answers.view_count AS answer_view_count
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` posts
LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` answers
ON posts.id = answers.parent_id
WHERE EXTRACT(YEAR FROM posts.creation_date) between 2010 and 2011 AND posts.answer_count >=5;
"""
response2 = stackOverflow.query_to_pandas_safe(query2,max_gb_scanned=70)

In [18]:
response2.shape

(1029773, 40)

In [14]:
response1.to_parquet('StackOverflow_test_2.parquet')  

In [7]:
query3 = """SELECT posts.id AS post_id,
posts.title AS post_title,
posts.body AS post_body,
posts.accepted_answer_id AS post_accepted_answer_id,
posts.answer_count AS post_answer_count,
posts.comment_count AS post_comment_count,
posts.community_owned_date AS post_community_owned_date,
posts.creation_date AS post_creation_date,
posts.favorite_count AS post_favorite_count,
posts.last_activity_date AS post_last_activity_date,
posts.last_edit_date AS post_last_edit_date,
posts.last_editor_display_name AS post_last_editor_display_name,
posts.last_editor_user_id AS post_last_editor_user_id,
posts.owner_display_name AS post_owner_display_name,
posts.owner_user_id AS post_owner_user_id,
posts.parent_id AS post_parent_id,
posts.post_type_id AS post_post_type_id,
posts.score AS post_score,
posts.tags AS post_tags,
posts.view_count AS post_view_count,
answers.id AS answer_id,
answers.title AS answer_title,
answers.body AS answer_body,
answers.accepted_answer_id AS answer_accepted_answer_id,
answers.answer_count AS answer_answer_count,
answers.comment_count AS answer_comment_count,
answers.community_owned_date AS answer_community_owned_date,
answers.creation_date AS answer_creation_date,
answers.favorite_count AS answer_favorite_count,
answers.last_activity_date AS answer_last_activity_date,
answers.last_edit_date AS answer_last_edit_date,
answers.last_editor_display_name AS answer_last_editor_display_name,
answers.last_editor_user_id AS answer_last_editor_user_id,
answers.owner_display_name AS answer_owner_display_name,
answers.owner_user_id AS answer_owner_user_id,
answers.parent_id AS answer_parent_id,
answers.post_type_id AS answer_post_type_id,
answers.score AS answer_score,
answers.tags AS answer_tags,
answers.view_count AS answer_view_count
FROM `bigquery-public-data.stackoverflow.stackoverflow_posts` posts
LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` answers
ON posts.id = answers.parent_id
WHERE EXTRACT(YEAR FROM posts.creation_date) >= 2012 AND posts.answer_count >=5;
"""
response3 = stackOverflow.query_to_pandas_safe(query3,max_gb_scanned=70)

In [8]:
response3.shape

(1214843, 40)

In [10]:
response3.to_parquet('StackOverflow_test.parquet')  

In [15]:
import pandas as pd

def get_df_size_gb(df):
    memory_usage_bytes = df.memory_usage(deep=True).sum()
    memory_usage_gb = memory_usage_bytes / (1024 ** 3)
    return memory_usage_gb

In [16]:
get_df_size_gb(response3)

3.071215070784092

In [19]:
get_df_size_gb(response2)

2.251500024460256

In [22]:
get_df_size_gb(response1)

1.7234860341995955

In [23]:
final_data = pd.concat([response1,response2,response3])

In [24]:
final_data.shape

(3024056, 40)

In [25]:
get_df_size_gb(final_data) #Final data is 7GB 
#Filters used: Stackoverflow posts created in or after 2008 that have atleast 5 answers

7.068731744773686

In [26]:
final_data.columns

Index(['post_id', 'post_title', 'post_body', 'post_accepted_answer_id',
       'post_answer_count', 'post_comment_count', 'post_community_owned_date',
       'post_creation_date', 'post_favorite_count', 'post_last_activity_date',
       'post_last_edit_date', 'post_last_editor_display_name',
       'post_last_editor_user_id', 'post_owner_display_name',
       'post_owner_user_id', 'post_parent_id', 'post_post_type_id',
       'post_score', 'post_tags', 'post_view_count', 'answer_id',
       'answer_title', 'answer_body', 'answer_accepted_answer_id',
       'answer_answer_count', 'answer_comment_count',
       'answer_community_owned_date', 'answer_creation_date',
       'answer_favorite_count', 'answer_last_activity_date',
       'answer_last_edit_date', 'answer_last_editor_display_name',
       'answer_last_editor_user_id', 'answer_owner_display_name',
       'answer_owner_user_id', 'answer_parent_id', 'answer_post_type_id',
       'answer_score', 'answer_tags', 'answer_view_count'],


In [27]:
final_data['post_creation_date'].min()

Timestamp('2008-07-31 21:42:52.667000+0000', tz='UTC')

In [28]:
final_data['post_creation_date'].max()

Timestamp('2016-06-11 20:35:20.063000+0000', tz='UTC')

### Copy parquet files to GCS

In [13]:
!gsutil -m cp /StackOverflow_test.parquet gs://msca-bdp-student-gcs/Group6

Copying file:///StackOverflow_test.parquet [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1/1 files][685.4 MiB/685.4 MiB] 100% Done  68.0 MiB/s ETA 00:00:00           
Operation completed over 1 objects/685.4 MiB.                                    


In [14]:
!gsutil -m cp /StackOverflow_test_2.parquet gs://msca-bdp-student-gcs/Group6

Copying file:///StackOverflow_test_2.parquet [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1/1 files][801.3 MiB/801.3 MiB] 100% Done  59.8 MiB/s ETA 00:00:00           
Operation completed over 1 objects/801.3 MiB.                                    


In [15]:
!gsutil -m cp /StackOverflow_test_3.parquet gs://msca-bdp-student-gcs/Group6

Copying file:///StackOverflow_test_3.parquet [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

- [1/1 files][597.5 MiB/597.5 MiB] 100% Done  59.5 MiB/s ETA 00:00:00           
Operation completed over 1 objects/597.5 MiB.                                    
